# MLOps Workshop: API-Entwicklung mit FastAPI

## Einführung
In diesem Notebook entwickeln wir eine REST-API mit FastAPI für unser Customer Churn Prediction Modell. Wir werden Best Practices für ML-Model-Serving implementieren, einschließlich Input-Validierung, Fehlerbehandlung und Performance-Optimierung.

## Lernziele
Nach Abschluss dieses Notebooks können Sie:
- Eine REST-API mit FastAPI entwickeln
- ML-Modelle über HTTP-Endpoints bereitstellen
- Input-Validierung und Fehlerbehandlung implementieren
- API-Tests schreiben und durchführen
- API-Performance überwachen

## Voraussetzungen
- Abgeschlossenes Modelltraining und -evaluierung (Notebooks 03 und 04)
- Grundlegendes Verständnis von HTTP und REST-APIs

## 1. Setup und Installation

Zuerst installieren wir die benötigten Bibliotheken:


In [ ]:
import mlflow
import pandas as pd
import numpy as np
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from typing import List, Dict
import uvicorn
from datetime import datetime
import joblib

# MLflow Setup
mlflow.set_tracking_uri("file:./mlruns")
client = mlflow.client.MlflowClient()

# Modell laden
model_name = "customer_churn_predictor"
model_version = client.get_latest_versions(model_name, stages=["Staging"])[0]
print(model_version.run_id)

model = mlflow.sklearn.load_model(f"./mlruns/707424938223910991/f72db19fc60a4fa4a0238b80aa60347c/artifacts/model") # Pfad bitte anpassen

## 2. API-Entwicklung

### Aufgabe 1: Input-Validierung mit Pydantic
Erstellen Sie Pydantic-Modelle für die API-Requests.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
# Pydantic models
class CustomerFeatures(BaseModel):
    """Pydantic model for customer features"""
    gender: int = Field(..., ge=0, le=1, description="Gender (0=male, 1=female)")
    tenure: float = Field(..., ge=0, description="Customer tenure in months")
    MonthlyCharges: float = Field(..., ge=0, description="Monthly charges in dollars")
    TotalCharges: float = Field(..., ge=0, description="Total charges in dollars")
    AvgCostPerService: float = Field(..., ge=0, description="Average cost per service")
    CustomerAge: float = Field(..., ge=0, le=120, description="Customer age in years")

    @validator('TotalCharges')
    def validate_total_charges(cls, v, values):
        """Validate that TotalCharges makes sense given MonthlyCharges and tenure"""
        if 'MonthlyCharges' in values and 'tenure' in values:
            if v > values['MonthlyCharges'] * values['tenure'] * 1.5:
                raise ValueError('TotalCharges seems unusually high given MonthlyCharges and tenure')
        return v

    class Config:
        schema_extra = {
            "example": {
                "gender": 1,
                "tenure": 12,
                "MonthlyCharges": 89.9,
                "TotalCharges": 1078.8,
                "AvgCostPerService": 29.97,
                "CustomerAge": 42
            }
        }

class PredictionRequest(BaseModel):
    """Pydantic model for batch prediction requests"""
    data: List[CustomerFeatures]

class PredictionResponse(BaseModel):
    """Pydantic model for prediction responses"""
    predictions: List[float]
    prediction_time: datetime
    model_version: str
    request_id: str
```
</details>

### Aufgabe 2: FastAPI-Anwendung erstellen
Implementieren Sie die FastAPI-Anwendung mit Endpoints.

<details>
<summary>👉 Lösung anzeigen</summary>
Check main.py
</details>


### Aufgabe 3: Middleware und Error Handling
Implementieren Sie Middleware für Logging und Error Handling.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
from fastapi import Request
from fastapi.middleware.cors import CORSMiddleware
import logging
import time

# Logging konfigurieren
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Middleware für Request-Logging
@app.middleware("http")
async def log_requests(request: Request, call_next):
    """
    Middleware für Request-Logging und Performance-Monitoring
    """
    start_time = time.time()
    
    # Request-Details loggen
    logger.info(f"Request: {request.method} {request.url}")
    
    # Request verarbeiten
    response = await call_next(request)
    
    # Performance-Metriken berechnen
    process_time = time.time() - start_time
    logger.info(f"Response time: {process_time:.3f} seconds")
    
    # Response-Header hinzufügen
    response.headers["X-Process-Time"] = str(process_time)
    
    return response

# CORS-Middleware hinzufügen
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In Produktion einschränken!
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Globaler Exception Handler
@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    """
    Globaler Exception Handler für einheitliche Fehlerbehandlung
    """
    logger.error(f"Global exception: {exc}")
    return JSONResponse(
        status_code=500,
        content={
            "error": str(exc),
            "timestamp": datetime.now().isoformat(),
            "path": str(request.url)
        }
    )
```
</details>

## 3. API-Tests

### Aufgabe 4: Unit-Tests erstellen
Implementieren Sie Unit-Tests für die API.

<details>
<summary>👉 Lösung anzeigen</summary>
Check main_test.py
</details>
